In [1]:
from collections import defaultdict
import cv2
import numpy as np
import os
from copy import deepcopy

from datetime import datetime

import torch
# import torch.nn as nn
# import torch.nn.functional as F

import albumentations as A

import matplotlib.pyplot as plt

# from datetime import datetime

import models
from timm.models import create_model

In [2]:
root = "/data/ephemeral/home/datasets/normal/VS_01.매장이동_01.매장이동/"

folder_name = "VS_01.매장이동_01.매장이동"
npy_root = "./npys/"

if not os.path.exists(npy_root):
    os.makedirs(npy_root)
if not os.path.exists(npy_root+folder_name):
    os.makedirs(npy_root+folder_name)

In [3]:
file_list = os.listdir(root)
print(f"==>> file_list: {file_list}")
file_list.sort()
print(f"==>> file_list: {file_list}")

==>> file_list: ['C_1_1_93_BU_DYB_10-19_11-51-57_CC_DF1_F4_F4.mp4', 'C_1_1_93_BU_DYB_10-19_11-51-57_CE_DF1_F4_F4.mp4', 'C_1_1_88_BU_DYA_07-18_16-17-30_d_DF1_F2_M2.mp4', 'C_1_1_88_BU_DYB_10-19_10-56-16_CD_DF1_F4_F4.mp4', 'C_1_1_92_BU_DYA_07-18_16-42-53_a_DF1_F2_F2.mp4', 'C_1_1_86_BU_DYA_07-18_16-00-04_f_DF1_F2_F2.mp4', 'C_1_1_92_BU_DYB_10-19_11-45-54_CE_DF1_F4_F4.mp4', 'C_1_1_89_BU_DYB_10-19_11-02-29_CB_DF1_F4_F4.mp4', 'C_1_1_90_BU_DYB_10-19_11-08-25_CC_DF1_F4_F4.mp4', 'C_1_1_93_BU_DYB_10-19_11-51-57_CA_DF1_F4_F4.mp4', 'C_1_1_92_BU_DYA_07-18_16-42-53_f_DF1_F2_F2.mp4', 'C_1_1_91_BU_DYA_07-18_16-36-43_b_DF1_F2_M2.mp4', 'C_1_1_93_BU_DYB_10-19_11-51-57_CB_DF1_F4_F4.mp4', 'C_1_1_86_BU_DYB_10-19_13-43-22_CB_DF1_M4_M4.mp4', 'C_1_1_87_BU_DYA_07-18_16-08-05_a_DF1_F2_M2.mp4', 'C_1_1_87_BU_DYB_10-19_13-49-35_CB_DF1_M4_M4.mp4', 'C_1_1_92_BU_DYB_10-19_11-45-54_CD_DF1_F4_F4.mp4', 'C_1_1_90_BU_DYA_07-18_16-30-36_c_DF1_F2_M2.mp4', 'C_1_1_90_BU_DYB_10-19_11-08-25_CD_DF1_F4_F4.mp4', 'C_1_1_90_BU_DYA_07-1

In [4]:
segments_num = 1
# 모델에 들어갈 frame수는 16 * segments_num

model = create_model(
    "vit_small_patch16_224",
    img_size=224,
    pretrained=False,
    num_classes=710,
    all_frames=16 * segments_num,
    # tubelet_size=args.tubelet_size,
    # drop_rate=args.drop,
    # drop_path_rate=args.drop_path,
    # attn_drop_rate=args.attn_drop_rate,
    # head_drop_rate=args.head_drop_rate,
    # drop_block_rate=None,
    # use_mean_pooling=args.use_mean_pooling,
    # init_scale=args.init_scale,
    # with_cp=args.with_checkpoint,
)

load_dict = torch.load(
    "/data/ephemeral/home/level2-3-cv-finalproject-cv-06/datapreprocess/vit_s_k710_dl_from_giant.pth"
)
# backbone pth 경로

model.load_state_dict(load_dict["module"])

model.to("cuda")
model.eval()

tf = A.Resize(224, 224)

In [20]:
# # test
# file_list = file_list[4:6]
# print(f"==>> file_list: {file_list}")

==>> file_list: ['C_1_1_85_BU_DYB_10-19_13-12-56_CF_DF1_M4_M4.mp4', 'C_1_1_86_BU_DYA_07-18_16-00-04_a_DF1_F2_F2.mp4']


In [5]:
batch_size = 16

# Loop through the video frames
for file_name in file_list:
    path = root + file_name

    time_start = datetime.now()

    print(f"{file_name} feature extracting starts")

    cap = cv2.VideoCapture(path)

    # 710차원 feature array 저장할 list
    np_list = []

    # 16 * segments_num 프레임씩 저장할 list
    frames = []
    frame_count = 0

    # input tensor 저장할 list
    input_list = []
    input_count = 0

    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()
        # frame.shape = (height, width, 3)

        frame_count += 1  # Increment frame count

        if success:
            frame = tf(image=frame)["image"]
            # frame.shape = (224, 224, 3)

            frame = np.expand_dims(frame, axis=0)
            # frame.shape = (1, 224, 224, 3)
            frames.append(frame.copy())

            if frame_count == 16 * segments_num:
                assert len(frames) == 16 * segments_num
                frames = np.concatenate(frames)
                # in_frames.shape = (16 * segments_num, 224, 224, 3)
                in_frames = frames.transpose(3, 0, 1, 2)
                # # in_frames.shape = (RGB 3, frame T=16 * segments_num, H=224, W=224)
                in_frames = np.expand_dims(in_frames, axis=0)
                # in_frames.shape = (1, 3, 16 * segments_num, 224, 224)
                in_frames = torch.from_numpy(in_frames).float()
                # in_frames.shape == torch.Size([1, 3, 16 * segments_num, 224, 224])

                input_list.append(in_frames.detach().clone())

                frame_count = 0
                frames = []

                input_count += 1

                if input_count == batch_size:
                    # input_batch.shape == torch.Size([batch_size, 3, 16 * segments_num, 224, 224])
                    input_batch = torch.cat(input_list, dim=0).to("cuda")

                    with torch.no_grad():
                        output = model(input_batch)
                        # output.shape == torch.Size([batch_size, 710])

                    np_list.append(output.cpu().numpy())

                    input_count = 0
                    input_list = []
        else:
            # 남은 프레임, input_list가 지정 개수에서 모자를 때 예외 처리
            if frame_count != 0:
                len_frames_left = 16 * segments_num - len(frames)
                # len_input_list_left = batch_size - len(input_list)
                for i in range(len_frames_left):
                    frames.append(frames[-1].copy())

                assert len(frames) == 16 * segments_num

                frames = np.concatenate(frames)
                # in_frames.shape = (16 * segments_num, 224, 224, 3)
                in_frames = frames.transpose(3, 0, 1, 2)
                # # in_frames.shape = (RGB 3, frame T=16 * segments_num, H=224, W=224)
                in_frames = np.expand_dims(in_frames, axis=0)
                # in_frames.shape = (1, 3, 16 * segments_num, 224, 224)
                in_frames = torch.from_numpy(in_frames).float()
                # in_frames.shape == torch.Size([1, 3, 16 * segments_num, 224, 224])

                input_list.append(in_frames.detach().clone())

                # assert len(input_list) == batch_size

                # input_batch.shape == torch.Size([batch_size, 3, 16 * segments_num, 224, 224])
                input_batch = torch.cat(input_list, dim=0).to("cuda")

                with torch.no_grad():
                    output = model(input_batch)
                    # output.shape == torch.Size([len(input_list), 710])

                np_list.append(output.cpu().numpy())

                frame_count = 0
                frames = []
                input_count = 0
                input_list = []

            # Break the loop if the end of the video is reached
            break

    file_outputs = np.concatenate(np_list)

    np.save((npy_root + folder_name + "/" + file_name), file_outputs)

    cap.release()

    time_end = datetime.now()
    total_time = time_end - time_start
    total_time = str(total_time).split(".")[0]

    print(f"{file_name} feature extracting ended. Elapsed time: {total_time}")

# cv2.destroyAllWindows()

C_1_1_85_BU_DYB_10-19_13-12-56_CB_DF1_M4_M4.mp4 feature extracting starts
C_1_1_85_BU_DYB_10-19_13-12-56_CB_DF1_M4_M4.mp4 feature extracting ended. Elapsed time: 0:00:06
C_1_1_85_BU_DYB_10-19_13-12-56_CC_DF1_M4_M4.mp4 feature extracting starts
C_1_1_85_BU_DYB_10-19_13-12-56_CC_DF1_M4_M4.mp4 feature extracting ended. Elapsed time: 0:00:05
C_1_1_85_BU_DYB_10-19_13-12-56_CD_DF1_M4_M4.mp4 feature extracting starts
C_1_1_85_BU_DYB_10-19_13-12-56_CD_DF1_M4_M4.mp4 feature extracting ended. Elapsed time: 0:00:05
C_1_1_85_BU_DYB_10-19_13-12-56_CE_DF1_M4_M4.mp4 feature extracting starts
C_1_1_85_BU_DYB_10-19_13-12-56_CE_DF1_M4_M4.mp4 feature extracting ended. Elapsed time: 0:00:05
C_1_1_85_BU_DYB_10-19_13-12-56_CF_DF1_M4_M4.mp4 feature extracting starts
C_1_1_85_BU_DYB_10-19_13-12-56_CF_DF1_M4_M4.mp4 feature extracting ended. Elapsed time: 0:00:05
C_1_1_86_BU_DYA_07-18_16-00-04_a_DF1_F2_F2.mp4 feature extracting starts
C_1_1_86_BU_DYA_07-18_16-00-04_a_DF1_F2_F2.mp4 feature extracting ended. Elap

In [37]:
feature_load = np.load(
    "/data/ephemeral/home/level2-3-cv-finalproject-cv-06/datapreprocess/npys/VS_01.매장이동_01.매장이동/C_1_1_85_BU_DYB_10-19_13-12-56_CF_DF1_M4_M4.mp4.npy"
)
print(f"==>> feature_load.shape: {feature_load.shape}")

==>> feature_load.shape: (57, 710)


In [40]:
feature_load[0] == feature_load[1]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [38]:
feature_load[0]

array([ 1.68963552e+00,  1.07660985e+00, -1.75942361e+00, -1.13592410e+00,
       -3.80988181e-01,  7.05899119e-01, -9.26711202e-01,  4.39998746e-01,
       -6.39206171e-02, -1.60322285e+00,  9.70086575e-01,  9.31643844e-02,
        1.10113621e+00,  1.43732023e+00,  1.97572216e-01, -3.23188305e-01,
       -5.82090080e-01, -1.05773067e+00,  9.98586595e-01,  1.82913888e+00,
       -1.89967334e-01,  1.25417411e+00,  6.22986197e-01, -7.53094018e-01,
        1.08373582e-01, -1.84132084e-01, -8.38344216e-01,  4.36990768e-01,
       -1.51385605e-01, -4.42820519e-01,  6.78801537e-02,  2.62016952e-01,
        7.86708653e-01, -8.41127038e-01, -3.60494614e-01,  1.02109194e+00,
        4.90810692e-01,  7.80149698e-02,  2.13103235e-01,  2.29127169e-01,
        9.19858098e-01,  2.38339126e-01, -2.63758898e-02, -1.69829413e-01,
       -1.48579001e+00, -9.71371531e-02,  7.01533914e-01, -1.28257275e-01,
       -7.37025619e-01,  3.63173425e-01, -4.46242988e-01,  2.32806087e-01,
        2.37338781e-01, -

In [39]:
feature_load[1]

array([ 1.63577175e+00,  1.09855938e+00, -1.77250195e+00, -1.11356091e+00,
       -3.93214345e-01,  7.11709797e-01, -8.86671484e-01,  4.69419837e-01,
       -5.69256544e-02, -1.55942500e+00,  9.67932343e-01,  4.60957214e-02,
        1.09285605e+00,  1.40178561e+00,  1.82727635e-01, -3.16147685e-01,
       -5.85943758e-01, -1.05143821e+00,  9.85271454e-01,  1.83332360e+00,
       -1.81374311e-01,  1.26979518e+00,  5.84595799e-01, -7.58406281e-01,
        1.14465803e-01, -1.86899558e-01, -8.21425319e-01,  4.19477642e-01,
       -1.58282667e-01, -4.81950581e-01,  8.38297158e-02,  2.99513578e-01,
        7.83126116e-01, -8.58671069e-01, -3.46607357e-01,  9.59519863e-01,
        4.48049545e-01,  7.96359777e-02,  2.05167592e-01,  2.34825552e-01,
        9.35081005e-01,  2.22574711e-01, -6.61726594e-02, -1.98212326e-01,
       -1.48041880e+00, -9.66389775e-02,  6.66265368e-01, -1.38680339e-01,
       -7.24636674e-01,  3.68397534e-01, -4.34812605e-01,  2.53913164e-01,
        2.46632636e-01, -